# Setup on TACC

In terminal:
- Go to home directory: `cd ~/`
- Make python venv: `python3 -m venv venv_39`
- Go to scratch directory: `cds`
- Clone brainlit: `git clone https://github.com/neurodata/brainlit.git`
- Install in editable mode `cd brainlit && pip install -e .`
- Make jupyter kernel for `venv_39` environment ([tutorial](https://queirozf.com/entries/jupyter-kernels-how-to-add-change-remove)) e.g. `pip install jupyter && ipython kernel install --name "venv_39-home" --user`

In Jupyter Notebook:
- Make sure kernel is the one you made for `venv_39` e.g. `venv_39-home`
- Run code below

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from brainlit.BrainLine.util import (
    json_to_points,
    download_subvolumes,
)
from brainlit.BrainLine.apply_ilastik import (
    ApplyIlastik,
    ApplyIlastik_LargeImage,
    plot_results,
    examine_threshold,
)
from brainlit.BrainLine.analyze_results import SomaDistribution
from brainlit.BrainLine.imports import *
import json
from matplotlib_scalebar.scalebar import ScaleBar
from cloudvolume import CloudVolume

# Soma Detection Example
### In this notebook we validate, process, and view results from a portion of an image

In [ ]:
brainlit_path = Path("/scratch/09423/tathey1/brainlit")
data_file = (
    brainlit_path / "docs" / "notebooks" / "pipelines" / "BrainLine" / "soma_data.json"
)
# Modify base path of test sample according to your system
with open(data_file, "r") as f:
    data = json.load(f)

data["brain2paths"]["test"]["base"] = (
    "precomputed://file://"
    + str(
        brainlit_path
        / "docs"
        / "notebooks"
        / "pipelines"
        / "BrainLine"
        / "example-data"
    )
    + "/"
)

brain2paths = data["brain2paths"]

with open(data_file, "w") as f:
    json.dump(data, f, indent=4)
    
brain = "test"  # brain ID
soma_data_dir = (
    str(
        brainlit_path
        / "docs"
        / "notebooks"
        / "pipelines"
        / "BrainLine"
        / "validation-data"
        / "soma"
    )
    + "/"
)  # path to directory where training/validation data should be stored
antibody_layer, background_layer, endogenous_layer = "antibody", "background", "endogenous"

# Download

In [ ]:
download_subvolumes(
    soma_data_dir,
    brain_id=brain,
    data_file=data_file,
    layer_names=[antibody_layer, background_layer, endogenous_layer],
    dataset_to_save="val",
)

# View [also possible in napari]

In [ ]:
fname = Path(soma_data_dir) / f"brain{brain}" / "val" / "2062_1949_1824_pos.h5"
scale = [1.8, 1.8, 2]  # voxel size in microns

with h5py.File(fname, "r") as f:
    im = f.get("image_3channel")
    image_fg = im[0, :, :, :]
    image_bg = im[1, :, :, :]
    image_endo = im[2, :, :, :]
    


plt.imshow(np.amax(image_fg, axis=0), cmap='gray', vmin=0, vmax=np.amax(image_fg)-5000)


scalebar = ScaleBar(0.3, "um")
plt.gca().add_artist(scalebar)
plt.show()

# Predict on Validation Data

In [ ]:
ilastik_project = str(
    brainlit_path
    / Path("experiments/BrainLine/data/models/soma/matt_soma_rabies_pix_3ch.ilp")
)  # path to ilastik model to be used
ilastik_path = "/scratch/09423/tathey1/ilastik-1.4.0-Linux/run_ilastik.sh"  # path to ilastik executable (see here: https://www.ilastik.org/documentation/basics/headless.html)
brains = [brain]

In [ ]:
applyilastik = ApplyIlastik(
    ilastik_path=ilastik_path,
    project_path=ilastik_project,
    brains_path=soma_data_dir,
    brains=brains,
)
applyilastik.process_subvols(ncpu=6)

In [ ]:
fig, _, _ = plot_results(
    data_dir=soma_data_dir,
    brain_ids=[brain],
    object_type="soma",
    positive_channel=0,
    doubles=[],
)
plt.show()

# Predict Full Image

In [ ]:
ilastik_largeimage = ApplyIlastik_LargeImage(
    ilastik_path=ilastik_path,
    ilastik_project=ilastik_project,
    data_file=data_file,
    results_dir=soma_data_dir + "brainr_results/",
    ncpu=1,
)
ilastik_largeimage.apply_ilastik_parallel(
    brain_id=brain,
    layer_names=[antibody_layer, background_layer, endogenous_layer],
    threshold=0.74,
    data_dir=soma_data_dir + "brainr_temp/",
    chunk_size=[100, 100, 100],
    min_coords=[2000, 1900, 1800],
    max_coords=[2100, 2000, 1900],
)

In [ ]:
results_txt = "/scratch/09423/tathey1/brainlit/docs/notebooks/pipelines/BrainLine/validation-data/soma/brainr_results/image_2000_1900_1800_somas.txt"
with open(results_txt) as f:
    lines = f.readlines()
print(f"{len(lines)-1} detected somas")

# View results

In [ ]:
detection_idx = 0
radius = 20

vol = CloudVolume("precomputed://file:///scratch/09423/tathey1/brainlit/docs/notebooks/pipelines/BrainLine/example-data/antibody", fill_missing=True)
coord = lines[detection_idx].strip().replace(" ","")[1:-1].split(",")
coord = [int(float(c)) for c in coord]
im = vol[coord[0]-radius:coord[0]+radius,coord[1]-radius:coord[1]+radius,coord[2]-radius:coord[2]+radius]

plt.imshow(np.amax(im, axis=0), cmap='gray', vmin=0, vmax = np.amax(im)-5000)
scalebar = ScaleBar(0.3, "um")
plt.gca().add_artist(scalebar)
plt.show()

# [Register and transform detected points with [CloudReg](https://cloudreg.neurodata.io/)]

# View Results [also possible in napari and brainrender]

In [ ]:
ontology_file = (
    brainlit_path / "brainlit" / "Brainline" / "data" / "ara_structure_ontology.json"
)
sd = SomaDistribution(
    brain_ids=["test"], data_file=data_file, ontology_file=ontology_file
)

In [ ]:
f = sd.napari_coronal_section(
    z=1073, subtype_colors={"test_type": "red"}, symbols=["o", "+", "^", "vbar"], fold_on=False, plot_type="plt"
)
plt.show()

In [ ]:
regions = [
    512,  # cerebellum
    688,  # cerebral cortex
    698,  # olfactory areas
    1089,  # hippocampal formation
    # 583, # claustrum
    477,  # striatum
    # 803, # pallidum
    351,  # bed nuclei of stria terminalis
    # 703, #cortical subplate
    1097,  # hypothalamus
    549,  # thalamus
    186,  # lateral habenula
    519,  # cerebellar nuclei
    313,  # midbrain
    1065,  # hindbrain
]  # allen atlas region IDs to be shown
# see: https://connectivity.brain-map.org/projection/experiment/480074702?imageId=480075280&initImage=TWO_PHOTON&x=17028&y=11704&z=3

composite_regions = {
    "Amygdalar Nuclei": [131, 295, 319, 780]
}  # Custom composite allen regions where key is region name and value is list of allen regions

sd.region_barchart(regions, composite_regions=composite_regions, normalize_region=512)

# Other visualization modes (when GUI environments are available e.g. locally)

## Napari

![napari](https://github.com/neurodata/brainlit/blob/develop/docs/notebooks/pipelines/BrainLine/images/napari-soma.png?raw=true)

## Brainrender

![brainrender](https://github.com/neurodata/brainlit/blob/develop/docs/notebooks/pipelines/BrainLine/images/brainrender-soma.png?raw=true)